# Comparação entre distâncias de isolamento

In [17]:
from tabulate import tabulate
from math import sqrt
from IPython.display import HTML, display
from normaslt import types, nbr5422, nbr1985, iec

In [18]:
obs = types.obs.LINHA_TRANSMISSAO
v1 = 550.
v2 = 69.
dra=0.9
h=nbr5422.umidAbs(dra=dra, t=25.)
nbr1985.distVertFT(du=v1, obs=obs)
h

20.649832888830474

In [19]:
kg=nbr5422.fatorKgFFFrenteLenta(types.gap.CONDUTOR_CRUZADO)
kcs=1.48 # risco 1e-6, 100 gaps em paralelo
dff = nbr5422.espacFFFrenteLenta(Us=800., Kcs=kcs, Fsfl=2.4, kafl=0.95, kg=kg, alpha=0.33)
dff

nan

In [20]:
# Cálculo do kafl real e ajuste do espaçamento
kafl=nbr5422.fatorAtmFrenteLenta(dra=dra, h=h, d=dff)
kafl
dff = nbr5422.espacFFFrenteLenta(Us=v1, Kcs=kcs, Fsfl=2, kafl=kafl, kg=kg)
dff

nan

## Distâncias elétricas para circuitos
NBR 5422 - Metodologia proposta

In [21]:
vnom = [69., 138., 230., 345., 440., 500., 765.]
vmax = [72.5, 145., 242., 362., 460., 550., 800.]
fsfl = [3., 3., 3., 3., 2.4, 2.4, 2.]
kg = 1.35
ka = 0.95
kcs = 1.48
dftff = []
dftfl = []
dffff = []
dfffl = []
for vn, vm, f in zip(vnom, vmax, fsfl):
  ftmo=vm/vn
  dftff.append(nbr5422.espacFTFreqFund(Us=vn, Ftmo=ftmo, kaff=ka, kgff=1.))
  dftfl.append(nbr5422.espacFTFrenteLenta(Us=vn, Kcs=kcs, Fsfl=f, kafl=ka, kg=kg))
  dffff.append(nbr5422.espacFFFreqFund(Us=vn, Ftmo=ftmo, kaff=ka, kgff=1.))
  dfffl.append(nbr5422.espacFFFrenteLenta(Us=vn, Kcs=kcs, Fsfl=f, kafl=ka, kg=kg, alpha=0.33))

table = zip(vnom, dftff, dftfl, dffff, dfffl)
display(HTML(tabulate(table, headers=['Vn', 'dftff', 'dftfl', 'dffff', 'dfffl'], tablefmt='html')))

/home/ck/programas/nbr5422/python/normaslt/nbr5422.py:58: UserWarning: Distância calculada fora da validade do modelo: 14 m.
  warn('Distância calculada fora da validade do modelo: %d m.' % d)
/home/ck/programas/nbr5422/python/normaslt/nbr5422.py:58: UserWarning: Distância calculada fora da validade do modelo: 16 m.
  warn('Distância calculada fora da validade do modelo: %d m.' % d)


Vn,dftff,dftfl,dffff,dfffl
69,0.171899,0.987253,0.27712,1.12074
138,0.3147,1.22093,0.517866,1.57342
230,0.500343,1.62072,0.847074,2.47342
345,0.733074,2.3093,1.28728,14.6664
440,0.930178,2.35961,1.68483,16.2586
500,1.1189,2.73542,2.08706,nan
765,1.69285,3.83804,3.44123,nan


Metodologia IEC

In [22]:
dftff = []
dftfl = []
dffff = []
dfffl = []
for vn, vm, f in zip(vnom, vmax, fsfl):
  ftmo=vm/vn
  dftff.append(iec. espacPowerFreq(un=vn/sqrt(3), ftmo=ftmo, fca=ka, k=kg))
  dffff.append(iec. espacPowerFreq(un=vn, ftmo=ftmo, fca=ka, k=kg))

table = zip(vnom, dftff, dffff)
display(HTML(tabulate(table, headers=['Vn', 'dftff', 'dffff'], tablefmt='html')))

Vn,dftff,dffff
69,0.099527,0.15895
138,0.179878,0.290333
230,0.280982,0.460103
345,0.402546,0.671283
440,0.501281,0.848742
500,0.592473,1.01751
765,0.852206,1.52447


## Distância entre circuitos paralelos
Ref. planilha Distâncias de Segurança Rev12.xls:Distâncias_fase_fase

In [6]:
def tabFsfl(v: int) -> float:
    # Fatores Fs_fl arbitrados na tabela
    match v:
        case 69 | 138 | 230 | 345:
            return 3.
        case 440 | 500:
            return 2.4
        case 765:
            return 2.

def tabKcs(v: int) -> float:
    # Fatores Kcs arbitrados na tabela
    match v:
        case 69 | 138 | 230 | 345:
            return 1.51
        case 440 | 500 | 765:
            return 1.48

vff = [
    [765, 69],
    [765, 138],
    [500, 69],
    [500, 138],
    [230, 69],
    [69, 138],
    [69, 345],
    [69, 440],
    [138, 230],
    [138, 345],
    [138, 440],
    [230, 345],
    [230, 440],
    [230, 500],
    [230, 765],
    [345, 440],
    [345, 500],
    [345, 765],
    [440, 500],
    [440, 765],
    [500, 765]
]

In [7]:
tvm=[]
tkcs=[]
tfsfl=[]
tkg=[]
tdffff=[]
tdfffl=[]
for v in vff:
  vm = max(v)
  kcs=tabKcs(vm)
  fsfl=tabFsfl(vm)
  kg=nbr5422.fatorKgFFFrenteLenta(types.gap.CONDUTOR_PARALELO)
  tdffff.append(nbr5422.espacFFFreqFund(Us=float(vm), Ftmo=1.1, kaff=0.9, kgff=kg))
  tdfffl.append(nbr5422.espacFFFrenteLenta(Us=float(vm), Kcs=kcs, Fsfl=fsfl, kafl=0.95, kg=kg))
  tvm.append(vm)
  tkcs.append(kcs)
  tfsfl.append(fsfl)
  tkg.append(kg)

table = zip(vff, tvm, tkcs, tfsfl, tkg, tdffff, tdfffl)
display(HTML(tabulate(table, headers=['V','Vn', 'Kcs', 'FsFl', 'kg', 'dff_ff', 'dff_fl'], tablefmt='html')))

V,Vn,Kcs,FsFl,kg,dff_ff,dff_fl
"[765, 69]",765,1.48,2,1.52,2.25133,nan
"[765, 138]",765,1.48,2,1.52,2.25133,nan
"[500, 69]",500,1.48,2.4,1.52,1.36512,nan
"[500, 138]",500,1.48,2.4,1.52,1.36512,nan
"[230, 69]",230,1.51,3,1.52,0.620294,2.22438
"[69, 138]",138,1.51,3,1.52,0.386797,1.47636
"[69, 345]",345,1.51,3,1.52,0.922932,3.71306
"[69, 440]",440,1.48,2.4,1.52,1.1884,3.71317
"[138, 230]",230,1.51,3,1.52,0.620294,2.22438
"[138, 345]",345,1.51,3,1.52,0.922932,3.71306


In [8]:
# Cálculo pela NBR 5422:1985, método alternativo
for v in vff:
  vm = v[0]+v[1]
  pu=tabFsfl(max(v))
  dff1 = nbr1985.distHorizFF(float(vm))
  dff2 = nbr1985.distHorizFFAlt(float(vm), pu)
  print(vm, dff1, dff2)

834 9.204407676193489 5.528953037057247
903 9.728807676193489 6.3123427803182715
569 7.190407676193488 3.9611964383128333
638 7.714807676193488 4.793914711453032
299 5.138407676193489 1.9658351420659779
207 4.439207676193488 1.064941592832497
414 6.012407676193488 3.3817600717219123
509 6.734407676193488 3.2896733377900764
368 5.662807676193489 2.7788911599301387
483 6.536807676193488 4.3726336282720855
578 7.258807676193489 4.066192532517563
575 7.236007676193489 5.8474904417211695
670 7.958007676193488 5.201408079121558
730 8.414007676193489 6.000859237637737
995 10.428007676193488 7.420448755292127
785 8.832007676193488 6.773325546264284
845 9.288007676193487 7.658158764099405
1110 11.32 8.90456073185179
940 10.010007676193489 9.1465721336855
1205 12.270000000000001 10.210911259037026
1265 12.870000000000001 11.072456956605922


In [34]:
pu

1.48